In [37]:
import os
import pandas as pd

# Import data from the O*NET database, at ISCO-08 occupation level.
# The original data uses a version of SOC classification, but the data we load here
# are already cross-walked to ISCO-08 using: https://ibs.org.pl/en/resources/occupation-classifications-crosswalks-from-onet-soc-to-isco/

# The O*NET database contains information for occupations in the USA, including
# the tasks and activities typically associated with a specific occupation.

task_data = pd.read_csv("C:\\Users\\X\\Downloads\\onet_tasks.csv")
# isco08 variable is for occupation codes
# the t_* variables are specific tasks conducted on the job

# read employment data from Eurostat
# These datasets include quarterly information on the number of workers in specific
# 1-digit ISCO occupation categories. (Check here for details: https://www.ilo.org/public/english/bureau/stat/isco/isco08/)

path:str = "C:\\Users\\X\\Downloads\\Eurostat_employment_isco.xlsx"

def read_excel_sheet(path:str = None, r:int = None) -> dict | None:
    if r == None or path == None:
        return None
    
    # dictionary of names
    d = {}
    for i in range(1, r + 1):
        d[f"isco{i}"] = pd.read_excel(io=path, sheet_name=f"ISCO{i}")
    
    return d

dic = read_excel_sheet(path, 9)

# This will calculate worker totals in each of the chosen countries.
total_Belgium = dic['isco1']["Belgium"] + dic['isco2']["Belgium"] + dic['isco3']["Belgium"] + dic['isco4']["Belgium"] + dic['isco5']["Belgium"] + dic['isco6']["Belgium"] + dic['isco7']["Belgium"] + dic['isco8']["Belgium"] + dic['isco9']["Belgium"]
total_Spain = dic['isco1']["Spain"] + dic['isco2']["Spain"] + dic['isco3']["Spain"] + dic['isco4']["Spain"] + dic['isco5']["Spain"] + dic['isco6']["Spain"] + dic['isco7']["Spain"] + dic['isco8']["Spain"] + dic['isco9']["Spain"]
total_Poland = dic['isco1']["Poland"] + dic['isco2']["Poland"] + dic['isco3']["Poland"] + dic['isco4']["Poland"] + dic['isco5']["Poland"] + dic['isco6']["Poland"] + dic['isco7']["Poland"] + dic['isco8']["Poland"] + dic['isco9']["Poland"]

In [ ]:
import pandas as pd

# Let's merge all these datasets. We'll need a column that stores the occupation categories:
dic['isco1']['ISCO'] = 1
dic['isco2']['ISCO'] = 2
dic['isco3']['ISCO'] = 3
dic['isco4']['ISCO'] = 4
dic['isco5']['ISCO'] = 5
dic['isco6']['ISCO'] = 6
dic['isco7']['ISCO'] = 7
dic['isco8']['ISCO'] = 8
dic['isco9']['ISCO'] = 9

# and this gives us one large file with employment in all occupations.
all_data = pd.concat([dic['isco1'], dic['isco2'], dic['isco3'], dic['isco4'], dic['isco5'], dic['isco6'], dic['isco7'], dic['isco8'], dic['isco9']], ignore_index=True)

# We have 9 occupations and the same time range for each, so we can add the totals by
# adding a vector that is 9 times the previously calculated totals

all_data["total_Belgium"] = pd.concat([total_Belgium]*9, ignore_index=True)
all_data["total_Spain"] = pd.concat([total_Spain]*9, ignore_index=True)
all_data["total_Poland"] = pd.concat([total_Poland]*9, ignore_index=True)

# And this will give us shares of each occupation among all workers in a period-country
all_data['share_Belgium'] = all_data['Belgium'] / all_data['total_Belgium']
all_data['share_Spain'] = all_data['Spain'] / all_data['total_Spain']
all_data['share_Poland'] = all_data['Poland'] / all_data['total_Poland']

In [39]:
# Now let's look at the task data. We want the first digit of the ISCO variable only
import pandas as pd
import numpy as np
import re

task_data["isco08_1dig"] = task_data["isco08"].astype(str).str[:1].astype(int)

# And we'll calculate the mean task values at a 1-digit level 
# (more on what these tasks are below)
aggdata = task_data.groupby(["isco08_1dig"]).mean()
aggdata = aggdata.drop(columns=["isco08"])

# We'll be interested in tracking the intensity of Non-routine cognitive analytical tasks
# Using a framework reminiscent of the work by David Autor.

#These are the ones we're interested in:
# Non-routine cognitive analytical
# 4.A.2.a.4 Analyzing Data or Information
# 4.A.2.b.2 Thinking Creatively
# 4.A.4.a.1 Interpreting the Meaning of Information for Others

#Let's combine the data.
combined = pd.merge(all_data, aggdata, left_on='ISCO', right_on='isco08_1dig', how='left')
# Traditionally, the first step is to standardise the task values using weights 
# defined by share of occupations in the labour force. This should be done separately
# for each country. Standardisation -> getting the mean to 0 and std. dev. to 1.
# Let's do this for each of the variables that interests us:



In [43]:
from scipy.stats import gmean

shares = ['Belgium', 'Poland', 'Spain']
std_shares = ["std_Belgium_t_4A2a4", "std_Poland_t_4A2a4", "std_Spain_t_4A2a4"]
combineds = ["t_4A2a4", "t_4A2b2", "t_4A4a1"]

def load_temp(share_c:list[str] = None, std_c:list[str] = None, combined_c:list[str] = None) -> None:

    for x in range(len(share_c)):
        for y in range(len(std_c)):
            for z in range(len(combined_c)):
                temp_mean=np.average(combined[combined_c[z]],weights=combined[share_c[x]])
                temp_sd=np.sqrt(np.average((combined[combined_c[z]]-temp_mean)**2,weights=combined[share_c[x]]))
                combined[std_c[y]]=(combined[combined_c[z]]-temp_mean)/temp_sd

load_temp(shares, std_shares, combineds)

In [ ]:
# The next step is to calculate the `classic` task content intensity, i.e.
# how important is a particular general task content category in the workforce
# Here, we're looking at non-routine cognitive analytical tasks, as defined
# by David Autor and Darron Acemoglu:

from scipy import stats
combined["Belgium_NRCA"] = combined["std_Belgium_t_4A2a4"] + combined["std_Belgium_t_4A2b2"] + combined["std_Belgium_t_4A4a1"]
combined["Poland_NRCA"] = combined["std_Poland_t_4A2a4"] + combined["std_Poland_t_4A2b2"] + combined["std_Poland_t_4A4a1"]
combined["Spain_NRCA"] = combined["std_Spain_t_4A2a4"] + combined["std_Spain_t_4A2b2"] + combined["std_Spain_t_4A4a1"]

# And we standardise NRCA in a similar way.

def combine(combs, shares, ):

    temp_mean = np.average(combined["Belgium_NRCA"], weights=combined["share_Belgium"])
    temp_sd = np.sqrt(np.average((combined["Belgium_NRCA"] - temp_mean)**2, weights=combined["share_Belgium"]))
    combined["std_Belgium_NRCA"] = (combined["Belgium_NRCA"] - temp_mean) / temp_sd

temp_mean = np.average(combined["Poland_NRCA"], weights=combined["share_Poland"])
temp_sd = np.sqrt(np.average((combined["Poland_NRCA"] - temp_mean)**2, weights=combined["share_Poland"]))
combined["std_Poland_NRCA"] = (combined["Poland_NRCA"] - temp_mean) / temp_sd

temp_mean = np.average(combined["Spain_NRCA"], weights=combined["share_Spain"])
temp_sd = np.sqrt(np.average((combined["Spain_NRCA"] - temp_mean)**2, weights=combined["share_Spain"]))
combined["std_Spain_NRCA"] = (combined["Spain_NRCA"] - temp_mean) / temp_sd


# Finally, to track the changes over time, we have to calculate a country-level mean
# Step 1: multiply the value by the share of such workers.
combined["multip_Spain_NRCA"] = combined["std_Spain_NRCA"] * combined["share_Spain"]
combined["multip_Belgium_NRCA"] = combined["std_Belgium_NRCA"] * combined["share_Belgium"]
combined["multip_Poland_NRCA"] = combined["std_Poland_NRCA"] * combined["share_Poland"]

# Step 2: sum it up (it basically becomes another weighted mean)

agg_Spain = combined.groupby(["TIME"])["multip_Spain_NRCA"].sum().reset_index()
agg_Belgium = combined.groupby(["TIME"])["multip_Belgium_NRCA"].sum().reset_index()
agg_Poland = combined.groupby(["TIME"])["multip_Poland_NRCA"].sum().reset_index()

# We can plot it now!

import matplotlib.pyplot as plt
plt.plot(agg_Poland["TIME"], agg_Poland["multip_Poland_NRCA"])
plt.xticks(range(0, len(agg_Poland), 3), agg_Poland["TIME"][::3])
plt.show()

plt.plot(agg_Spain["TIME"], agg_Spain["multip_Spain_NRCA"])
plt.xticks(range(0, len(agg_Spain), 3), agg_Spain["TIME"][::3])
plt.show()

plt.plot(agg_Belgium["TIME"], agg_Belgium["multip_Belgium_NRCA"])
plt.xticks(range(0, len(agg_Belgium), 3), agg_Belgium["TIME"][::3])
plt.show()

# If this code gets automated and cleaned properly,
#  you should be able to easily add other countries as well as other tasks.
# E.g.:

# Routine manual
# 4.A.3.a.3	Controlling Machines and Processes
# 4.C.2.d.1.i	Spend Time Making Repetitive Motions
# 4.C.3.d.3	Pace Determined by Speed of Equipment


KeyError: 'std_Belgium_t_4A2b2'